<a href="https://colab.research.google.com/github/Vansik4/MLOpsReviews_PI/blob/main/desarrollo_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3195809 sha256=1fde76b1aa5095ae3106364c97db4e50672c02ac158b4f43a9229ff7d7140ce9
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split 
from surprise import accuracy
from surprise import KNNBasic
from surprise.model_selection import KFold
from surprise import SVD


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Dataset"
import pandas as pd
data = pd.read_csv("plataformas_ratings.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Dataset


In [ ]:
df1 = data.copy()
df1 = df1.drop(['show_id','type','director','cast','country','date_added','release_year','rating_x','listed_in','duration_int','duration_type','Plataforma'], axis=1)
df1.head()

,id,title,description,score_mean
0,as1,the grand seduction,a small fishing village must procure a local d...,3.47
1,as2,take care good night,a metro family decides to fight a cyber crimin...,3.55
2,as3,secrets of deception,after a man discovers his wife is cheating on ...,3.50
3,as4,pink: staying true,"pink breaks the mold once again, bringing her ...",3.54
4,as5,monster maker,teenage matt banting wants to work with a famo...,3.48


In [ ]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df1[['id', 'title', 'score_mean']], reader)
trainset, testset = train_test_split(data, test_size=.5)
kf = KFold(n_splits=5)
algo = SVD(n_factors=5)
for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.0485
RMSE: 0.0486
RMSE: 0.0488
RMSE: 0.0486
RMSE: 0.0488


In [ ]:
titulo = 1
df_titulo = df1[(df1['id'] == titulo) & (df1['score_mean'] > 0)]
df_titulo = df_titulo.reset_index(drop=True)

In [ ]:
recomendaciones_tituto = df1[['title','id']].iloc[:4499].copy()
recomendaciones_tituto.index = recomendaciones_tituto.id
recomendaciones_tituto = recomendaciones_tituto.drop(['id'], axis=1)
print(recomendaciones_tituto.shape)
recomendaciones_tituto.head()

(4499, 1)


,title
id,
as1,the grand seduction
as2,take care good night
as3,secrets of deception
as4,pink: staying true
as5,monster maker


In [ ]:
recomendaciones_tituto = recomendaciones_tituto.reset_index()
recomendaciones_tituto.head()

,id,title
0,as1,the grand seduction
1,as2,take care good night
2,as3,secrets of deception
3,as4,pink: staying true
4,as5,monster maker


In [ ]:
recomendaciones_tituto['score_mean'] = recomendaciones_tituto['id'].apply(lambda x: algo.predict(titulo, x).est)
recomendaciones_tituto = recomendaciones_tituto.sort_values('score_mean', ascending=False)
recomendaciones_tituto.head(5)

,id,title,score_mean
0,as1,the grand seduction,3.533609
2997,as2998,born too white,3.533609
3003,as3004,bookaboo uk,3.533609
3002,as3003,boonie bears: blast into the past,3.533609
3001,as3002,bordertown: laredo,3.533609
